In [4]:
from glob import glob
import pandas as pd
from datasets import Dataset, load_from_disk
from datasets import DatasetDict
import datasets

import pickle 

In [5]:
path = "/home/aisg/peerat/Thai_local_benchmark/datasets/*.csv"

In [6]:
food_name = {
    "north": ["ลาบเหนือ",
            "ขนมจีนน้ำเงี้ยว",
            "แกงฮังเล",
            "น้ำพริกหนุ่ม",
            "น้ำพริกอ่อง",
            "แกงหัวปลี",
            "แกงหยวก",
            "ไส้อั๋ว",
            "แคบหมู",
            "แกงกระด้าง"],
    "east": ["แจ่วบักเขือ",
            "อ่อมน้องวัว",
            "ตำบักหุ่ง",
            "หมกหน่อไม้",
            "ซอยจุ๊",
            "แกงขี้เหล็ก",
            "น้ำตกคอหมูย่าง",
            "ปลาส้ม",
            "แจ่วฮ้อน",
            "ตับหวาน"],
    "south": ["แก๋งพุงปลา",
            "แกงส้ม",
            "ลอกอผัดไข่",
            "ข้าวผั๊ดเคย",
            "แกงส้มปล๊าพงยอดพร๊าว",
            "ข้าวผั๊ดเคย",
            "ปล่าซายท๋อดขี้มิ้น",
            "ผั๊ดผั๊กเหลี๊ยง",
            "แก๋งเลียง",
            "ผั๊ดหลูกต๋อ"],
    "center": ["ผัดกระเพราหมูสับไข่ดาว", "ต้มยำกุ้ง", "ผัดไทกุ้งสด", "แกงเขียวหวานไก่", "ต้มข่าไก่",
                "ต้มจืด",
                "พะโล้",
                "ทอดมันปลากราย",
                "เต้าเจี้ยวหลน",
                "พะแนง"]
}

conversation_list = [
    "ประเพณีท้องถิ่น",
    "อาชีพ",
    "การคมนาคม",
    "ครอบครัว",
    "กลับบ้านช่วงเทศกาล",
    "การเมืองท้องถิ่น",
    "สภาพอากาศ",
    "อาหารท้องถิ่น",
    "เครื่องแต่งกายท้องถิ่น",
    "การศึกษาท้องถิ่น",
]

In [9]:
local_data = DatasetDict()

for local in ["east","north","south","center"]:
    task_data = DatasetDict()
    for file in glob(path):
        task_name = file.split("/")[-1].split(".")[0]
        df = pd.read_csv(file)
        if task_name == 'qa':
            new_df = df[["id",f"question_{local}",f'text_2_{local}',f'context_{local}']]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"text_2_{local}", "text_2")
            dataset = dataset.rename_column(f"question_{local}", "question")
            dataset = dataset.rename_column(f"context_{local}", "context")
            task_data[task_name] = dataset
        elif task_name == 'summarization':
            new_df = df[["id",f"text_2_{local}","text_1"]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"text_2_{local}", "text_2")
            task_data[task_name] = dataset
        elif 'translation' in task_name:
            task_name = f'translation_eng_{local}'
            new_df = df[["english",local]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"english", "text_1")
            dataset = dataset.rename_column(local, "text_2")
            task_data[task_name] = dataset

            task_name = f'translation_{local}_eng'
            new_df = df[["english",local]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"english", "text_2")
            dataset = dataset.rename_column(local, "text_1")
            task_data[task_name] = dataset


    # conversation
    task_name='conversation'
    new_df = pd.DataFrame(list(zip(list(range(len(conversation_list))), conversation_list, "."*len(conversation_list))),columns =['id', 'text_1',"text_2"])
    dataset = Dataset.from_pandas(new_df)
    task_data[task_name] = dataset

    task_name='food'
    food_list = food_name[local]
    new_df = pd.DataFrame(list(zip(list(range(len(food_list))), food_list, "."*len(food_list))),columns =['id', 'text_1', "text_2"])
    dataset = Dataset.from_pandas(new_df)
    task_data[task_name] = dataset

    local_data[local] = task_data

In [10]:
with open('datasets/dataset.pkl', 'wb') as file:  
    # A new file will be created 
    pickle.dump(local_data, file) 

In [11]:
with open('datasets/dataset.pkl', 'rb') as file: 
      
    # Call load method to deserialze 
    loaded_file = pickle.load(file) 
  
    print(loaded_file) 

DatasetDict({
    east: DatasetDict({
        qa: Dataset({
            features: ['id', 'question', 'text_2', 'context'],
            num_rows: 20
        })
        summarization: Dataset({
            features: ['id', 'text_2', 'text_1'],
            num_rows: 20
        })
        translation_eng_east: Dataset({
            features: ['text_1', 'text_2'],
            num_rows: 20
        })
        translation_east_eng: Dataset({
            features: ['text_2', 'text_1'],
            num_rows: 20
        })
        conversation: Dataset({
            features: ['id', 'text_1', 'text_2'],
            num_rows: 10
        })
        food: Dataset({
            features: ['id', 'text_1', 'text_2'],
            num_rows: 10
        })
    })
    north: DatasetDict({
        qa: Dataset({
            features: ['id', 'question', 'text_2', 'context'],
            num_rows: 20
        })
        summarization: Dataset({
            features: ['id', 'text_2', 'text_1'],
            num_ro

In [8]:
loaded_file['north']['translation_eng_north'][1]

{'text_1': "I rode my motorcycle to buy coffee at the entrance of the alley, only to find that my favorite shop was closed. So, I went to try a new place instead. It wasn't good at all. I wouldn't go buy it there again.",
 'text_2': 'ขับรถเครื่องไปซื้อกาแฟที่หน้าปากซอย แต่ก็พบว่าฮ้านประจำตี้ชอบกิ๋นมันปิดไปแล้ว ก่อเลยขับรถไปซื้อฮ้านใหม่ บ่อลำเลย จะบ่ไปซื้อแหมแล้ว'}